In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from numpy.random import seed
from tensorflow.keras import backend as K

2021-10-03 23:32:59.619738: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [3]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [4]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [5]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:26<00:00, 117.16it/s]


In [6]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 128.50it/s]


In [7]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [8]:
train=pd.concat(train_dt)

In [9]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:06<00:00, 502.45it/s]


In [10]:
test=pd.concat(test_dt)

In [11]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 886.61it/s]


In [12]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [13]:
scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [14]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_series(data):
    ids = data['id'].unique()
    id_data = data.groupby('id')
    series_data = []

    for i in ids:
        df = id_data.get_group(i)
        df = df.drop(['id', 'time'], axis=1)
        series_data.append(df.to_numpy())

    series_data = np.array(series_data)
    return series_data


def ids_26(label):
    mask = label['label'] == 26
    ids = label.loc[mask, 'id'].tolist()
    
    return ids


def make_split_dataset(series_train, ids, labels):
    final_list = []
    
    k_split = KFold(n_splits=15, shuffle=True, random_state=42)
    
    temp = np.array(range(3125))
    except_train_mask = np.setdiff1d(temp, ids, assume_unique=True)
    except_train = series_train[except_train_mask]
    except_label = labels[except_train_mask]
    
    train_26 = series_train[ids]

    for _, fold in k_split.split(train_26):
        temp_train = train_26[fold]
        temp_label = np.array([26] * len(temp_train))
        
        temp_train = np.concatenate([temp_train, except_train], axis=0)
        temp_label = np.concatenate([temp_label, except_label], axis=0)
        print(temp_train.shape)
        final_list.append([temp_train, temp_label])
        
    return final_list

In [15]:
series_train = make_series(train_sc)
series_test = make_series(test_sc)

In [16]:
ids = ids_26(train_label)
data_list = make_split_dataset(series_train, ids, train_label['label'])

(1709, 600, 18)
(1709, 600, 18)
(1709, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)
(1708, 600, 18)


In [17]:
import tensorflow as tf

In [18]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
    cat_y = tf.keras.utils.to_categorical(labels)

    BATCH_SIZE = 64
    train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000, seed=42)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return train_dataset

def make_val(series_data, labels):
    cat_y = tf.keras.utils.to_categorical(labels)

    BATCH_SIZE = 64
    val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return val_dataset

In [19]:
from tensorflow import keras

In [25]:
def focal(labels, y_pred, gamma=2):
    #y_pred = tf.nn.softmax(logits, dim=-1)  # [batch_size, num_classes]
    #labels = tf.squeeze(labels)  # label example: [0,1,2,3]
    #labels = tf.to_float(tf.one_hot(labels, depth=y_pred.shape[1]))

    loss = -labels * ((1 - y_pred) ** gamma) * K.log(y_pred)
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss


# 모델을 만들어 주는 함수
# 기존 base에서 overfitting이 심해, dropout을 늘림(아직 제출은 안해봄)
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.GRU(250, return_sequences=True, dropout=0.2, input_shape=[600, 18]),
            keras.layers.GRU(150, return_sequences=True, dropout=0.2),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [26]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'random_sampling.hdf5'
checkpoint_dir_path = os.path.join('checkpoint')
checkpoint_path = os.path.join('checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
    os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5)
]

In [27]:
models = []
for series in data_list:
    x_train, x_val, y_train, y_val = train_test_split(series[0], series[1], train_size=0.7, stratify=series[1], random_state=42)
    
    train_dataset = make_train(x_train, y_train)
    val_dataset = make_val(x_val, y_val)
    model = base()

    model.fit(train_dataset, validation_data = val_dataset, epochs=1000, callbacks=callbacks_list)
    model.load_weights(checkpoint_path)
    model.evaluate(train_dataset)
    model.evaluate(val_dataset)

    models.append(model)

Epoch 1/1000


2021-10-03 23:35:17.001896: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-10-03 23:35:17.141761: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


19/19 [==============================] - 1s 77ms/step - loss: 3.8051 - accuracy: 0.0736 - val_loss: 3.5003 - val_accuracy: 0.1111
Epoch 2/1000
19/19 [==============================] - 1s 57ms/step - loss: 3.3896 - accuracy: 0.1421 - val_loss: 3.2195 - val_accuracy: 0.1891
Epoch 3/1000
19/19 [==============================] - 1s 58ms/step - loss: 3.1173 - accuracy: 0.2015 - val_loss: 3.0105 - val_accuracy: 0.2534
Epoch 4/1000
19/19 [==============================] - 1s 57ms/step - loss: 2.8992 - accuracy: 0.2492 - val_loss: 2.7660 - val_accuracy: 0.2904
Epoch 5/1000
19/19 [==============================] - 1s 57ms/step - loss: 2.6858 - accuracy: 0.3018 - val_loss: 2.6031 - val_accuracy: 0.3275
Epoch 6/1000
19/19 [==============================] - 1s 58ms/step - loss: 2.4954 - accuracy: 0.3261 - val_loss: 2.4547 - val_accuracy: 0.3509
Epoch 7/1000
19/19 [==============================] - 1s 57ms/step - loss: 2.3095 - accuracy: 0.3863 - val_loss: 2.2989 - val_accuracy: 0.3782
Epoch 8/1000

KeyboardInterrupt: 

In [89]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
    pred = model.predict(series_test)
    pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [104]:
# 제출물 생성
sub.iloc[:, 1:] = pred
sub.to_csv('overfit.csv', index=False)